In [1]:
from langchain_community.document_loaders import PyPDFLoader 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema.prompt_template import format_document
from langchain.schema import StrOutputParser
import os

In [2]:
os.environ["GOOGLE_API_KEY"] = open('passgooglestudio.txt', 'r').read()

https://www.bcb.gov.br/publicacoes/atascopom

In [3]:
# Coleta de dados ----
pdf = PyPDFLoader("https://www.bcb.gov.br/content/copom/atascopom/Copom261-not20240320261.pdf")
ata = pdf.load_and_split()


# Definição do modelo LLM e Engenharia de prompt ----
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

doc_prompt = PromptTemplate.from_template("{page_content}") # extrai texto do PDF

llm_prompt = PromptTemplate.from_template("""
Você é um economista chefe experiente em análise de política monetária, tendo
trabalhado em fundos de investimentos e grandes bancos, além de ser PhD em
economia.

Sua tarefa é ler a ata a seguir da Reunião do Comitê de Política Monetária (Copom)
do Banco Central do Brasil e classificar o documento em hawkish ou dovish:

"{ata}"

Classificação:""") # prompt para o Gemini

# Combinar prompt, texto da ata e modelo
classificador = (
    {"ata": lambda ata: "\n\n".join(
      format_document(pagina, doc_prompt) for pagina in ata
      )}
    | llm_prompt
    | llm
    | StrOutputParser()
)

classificador.invoke(ata[2:8])

'Hawkish'

In [4]:
#  Bibliotecas ----
# from shiny import App, reactive, render, ui
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
# import os

In [5]:
os.environ["OPENAI_API_KEY"] = open('passopenai.txt', 'r').read()

In [16]:
pdf = PyPDFLoader('https://aluno.analisemacro.com.br/download/59224/?tmstv=1710776237')
                  #https://www.dynamo.com.br/uploads/a615cb4e89402e053db8cf63256714814c312510.pdf')
texto = pdf.load_and_split()

llm = ChatOpenAI(temperature = 0, model_name = "gpt-3.5-turbo")


llm_chain = LLMChain(llm = llm, prompt = prompt)
modelo = StuffDocumentsChain(
            llm_chain = llm_chain,
            document_variable_name = "TEXTO"
            )

resultado = modelo.run(texto)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}